In [1]:
import threading
import time
import logging

In [2]:
def consume(cond):
    print('consume 스레드 시작')
    
    with cond:
        print('consume 스레드 대기')
        cond.wait()
        print('consume 스레드 리소스 사용 완료')

In [3]:
def produce(cond):
    print('produce 스레드 시작')
    
    with cond:
        print('produce 스레드 리소스 생성')
        print('produce 스레드 작업 재개 신호 발생')
        cond.notifyAll()

In [4]:
if __name__ == '__main__':
    cond = threading.Condition()
    con1 = threading.Thread(name = 'consumer1', target = consume, args = (cond,))
    con2 = threading.Thread(name = 'consumer2', target = consume, args = (cond,))    
    prod = threading.Thread(name = 'producer1', target = produce, args = (cond,))    
    
    con1.start()
    #time.sleep(5)
    con2.start()
    #time.sleep(5)
    prod.start()

consume 스레드 시작
consume 스레드 대기
consume 스레드 시작
consume 스레드 대기
produce 스레드 시작
produce 스레드 리소스 생성
produce 스레드 작업 재개 신호 발생
consume 스레드 리소스 사용 완료
consume 스레드 리소스 사용 완료


In [5]:
# A 스레드는 2의 배수를 3개 모을때마다 정지
# B 스레드는 3의 배수를 5개 모을때마다 정지
# 각 루프를 10번 돌면서 결과를 출력

def A_thread(cond):
    global A_flag
    global A_fin
    
    A_flag = False
    A_fin = False
    
    print("A 스레드 시작")
    cnt = 0
    
    for i in range(10):
        with cond:
            while True:
                cnt += 1
                
                print("{0}".format(cnt * 2))
                
                if (not(cnt % 3)):
                    break
                    
            time.sleep(1)
        
            print("A 스레드 대기")
            A_flag = True
            cond.wait()
            A_flag = False
            print("A 스레드 재시작")
            
    A_fin = True
    print("A Thread 종료")

def B_thread(cond):
    global B_flag
    global B_fin
    
    B_flag = False
    B_fin = False

    print("B 스레드 시작")
    cnt = 0
    
    for i in range(10):
        with cond:
            while True:
                cnt += 1
                
                print("{0}".format(cnt * 3))
                
                if (not(cnt % 5)):
                    break
                    
            time.sleep(1)
                      
            print("B 스레드 대기")
            B_flag = True
            cond.wait()
            B_flag = False
            print("B 스레드 재시작")
            
    B_fin = True
    print("B Thread 종료")
            
def manager(cond):
    print("manager 스레드 시작")
    
    while True:
        with cond:
            print("A_flag: {0}, B_flag: {1}".format(A_flag, B_flag))
            print("A_fin: {0}, B_fin: {1}".format(A_fin, B_fin))
            # 현재 A도 대기, B도 대기중인지 체크
            if (A_flag or B_flag):
                #print("A, B 스레드 작업 재개 신호 발생")
                cond.notifyAll()
                
            if (A_fin and B_fin):
                break
                
            time.sleep(1)
            
    print("manager 종료")
        
if __name__ == '__main__':
    cond = threading.Condition()
    A = threading.Thread(name = 'A', target = A_thread, args = (cond,))
    B = threading.Thread(name = 'B', target = B_thread, args = (cond,))
    man = threading.Thread(name = 'manager', target = manager, args = (cond,))
    
    A.start()
    B.start()
    man.start()

A 스레드 시작
2
4
6
B 스레드 시작
manager 스레드 시작
A 스레드 대기
3
6
9
12
15
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
18
21
24
27
30
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
8
10
12
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
33
36
39
42
45
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
48
51
54
57
60
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
14
16
18
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
63
66
69
72
75
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
20
22
24
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
78
81
84
87
90
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_f

In [6]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json, requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import threading
import time
import logging

In [ ]:
# 크롤링 스레드 <-> 기록 스레드
# 크롤링 스레드는 24시간마다 크롤링을 진행함 (30초)
# 이전에 크롤링이 완료되었다면 다음날 혹은 다음 시간까지 대기
# 기록 스레드는 크롤링이 완료된 경우에만 기록을 진행함
# 혹은 주어진 시간이 만료된 경우 해당 정보를 기록함

CRAWLED_RESULT_SAVED_FILE = "res/crawl_data/result"

def crawl_thread(cond, text):
    global crawl_fin
    crawl_fin = False
    
    print("크롤링 스레드 시작")
    
    url = "https://www.youtube.com/results?search_query="

    driver = webdriver.Chrome(r"C:\chromedriver.exe")

    search_url = url + text
    
    while True:
        with cond:
            cnt = 0
            driver.get(url = search_url)

            body = driver.find_element_by_tag_name('body')

            global hrefURL
            hrefURL = set()

            shelfContents = driver.find_elements_by_css_selector(
                "#contents > ytd-shelf-renderer > div > div > ytd-vertical-list-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
            )

            for item in shelfContents:
                if (item != ''):
                    hrefURL.add(item.get_attribute('href'))

            while cnt < 50:
                body.send_keys(Keys.END)
                cnt += 1

            allContents = driver.find_elements_by_css_selector(
                "#contents > ytd-item-section-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
            )

            for item in allContents:
                if (item != ''):
                    hrefURL.add(item.get_attribute('href'))

            print(hrefURL)
            print("크롤링 개수: ", len(hrefURL))

            crawl_fin = True
            cond.wait()
            crawl_fin = False

def record_thread(cond):
    cnt = 0
    
    while True:
        if crawl_fin:
            f = open(CRAWLED_RESULT_SAVED_FILE, 'wb')
            pickle.dump(hrefURL, f)
            f.close()
            
        time.sleep(1);
            
def manager(cond):  
    while True:
        with cond:
            if crawl_fin:
                cond.notifyAll()
                
            time.sleep(1)
        
if __name__ == '__main__':
    cond = threading.Condition()
    A = threading.Thread(name = 'Crawler', target = crawl_thread, args = (cond, "시고르자브종"))
    B = threading.Thread(name = 'Recorder', target = record_thread, args = (cond,))
    man = threading.Thread(name = 'manager', target = manager, args = (cond,))
    
    A.start()
    time.sleep(2)
    B.start()
    man.start()

In [ ]:
import pickle

f = open(CRAWLED_RESULT_SAVED_FILE, 'rb')
hrefURL = pickle.load(f)
f.close()

print(hrefURL)